In [ ]:
try:
  %numpy_version  1.20.3
except:
   pass
!pip install bert-embedding
from bert_embedding import BertEmbedding

In [ ]:
!pip install nltk

In [ ]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import gensim
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from nltk.stem import PorterStemmer
from gensim.models import Word2Vec,TfidfModel, LdaModel
from gensim.corpora import Dictionary
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# load all files

In [ ]:
def load_dict():
    train_data = {}
    test_data = {}
    dictionary = {}
    tree = ET.parse('drive/MyDrive/NLP/hw3/dictionary.xml')
    root = tree.getroot()
    # ambiguose words
    for element in root:
            word_pos = element.attrib['item']
            train_data[element.attrib['item'][:-2]] = {"pos": element.attrib['item'][-1]}
            test_data[element.attrib['item'][:-2]] = {"pos": element.attrib['item'][-1]}
            dictionary[element.attrib['item'][:-2]] = {"pos": element.attrib['item'][-1]}

    # various senses for each word
    for word in train_data.keys():
        for sense in root.findall("./lexelt[@item='{}']/sense".format(word + "." + train_data[word]["pos"])):
            train_data[word][sense.attrib['id']] = []
            test_data[word][sense.attrib['id']] = []
            dictionary[word][sense.attrib['id']] = sense.attrib
    return dictionary, train_data, test_data


def preprocessing(text):
    punctuation = ['<','>',',','.','&','!','@','#','$','%','^','*','(',')','-','_','+','=',';','\'','/','\\','{','}','[',']','?',':','"']
    stopwords_list = stopwords.words('english')
    text = text.lower()
    text_num = re.sub('[0-9]+', '', text)
    words = word_tokenize(text_num)
    filtered_words = [word for word in words if word not in punctuation]
    filtered_words = [word for word in filtered_words if word not in stop_words]
    return filtered_words


def load_data(dataset, datapath):
    train_data = pd.read_csv(datapath)
    max_sequence_len = 0
    all = []
    for index, word_data in train_data.iterrows():
            context = word_data["context"]
            before_text = context[0:context.index("<head>")]
            after_text = context[context.index("</head>") + 7:]
            before = 9
            after = 10
            context_b = preprocessing(before_text)
            context_a = preprocessing(after_text)
            if len(context_b) < 9:
                before = len(context_b)
            if len(context_a) < 10:
                after = len(context_a)
            context = context_b[-before:] + [word_data['word']]
            context += context_a[:after]
            dataset[word_data['word']][word_data["sense_id"]].append(context)
            all.append(context)
    return dataset, all
    

fill train and test

In [ ]:
    def train_xy(dictionary,train):
        train_X = {}
        data_train, train_all = load_data(train, "drive/MyDrive/NLP/hw3/train.csv")
        for word in data_train.keys():
            X = []
            Y = []
            y = list(dictionary[word].keys())
            for sense in data_train[word].keys():
                if sense != "pos":
                    for vec in data_train[word][sense]:
                        X.append(vec)
                        Y.append(y.index(sense))
            train_X[word] = {"x": X, "y": Y}
        return train_X

    def test_xy(dictionary,test):
        test_X = {}
        data_test, test_all = load_data(test, "drive/MyDrive/NLP/hw3/test.csv")
        for word in data_test.keys():
            X = []
            Y = []
            ys = list(dictionary[word].keys())
            for sense in data_test[word].keys():
                if sense != "pos":
                    for vec in data_test[word][sense]:
                        X.append(vec)
                        Y.append(ys.index(sense))
            test_X[word] = {"x": X, "y": Y}
      return test_X

tf_idf weighting

In [ ]:
    def tfidf_weigth(mode, words_array, tfidf, words):
        s = 0
        if mode == 0:
            result = np.zeros(300)
        else:
            result = np.zeros(768)
        tf_idf_list = tfidf[words]
        i = 0
        for idf in tf_idf_list:
            for i, tfidf in idf:
                try:
                    result += tfidf * np.array(words_array[i])
                except ValueError:
                    print(words_array[i].shape, tfidf, result.shape)
                s += tfidf
        result /= s
        return result

word2vec

In [ ]:
def word2vec_representation(dataset, docs, mode,dict):
        data = {}
        dictionary = Dictionary(docs)
        BoW_corpus = [dictionary.doc2bow(text) for text in docs]
        for word in dataset.keys():
            labels = list(dict[word].keys())
            embeddings = []
            new_labels = []
            for sense in dataset[word].keys():
                if sense != "pos":
                    for context in dataset[word][sense]:
                        index = context.index(word)
                        h = index - 3
                        t = index + 4
                        if index - 3 < 0:
                            h = 0
                        if index + 4 > len(context):
                            t = len(context)
                        bow_words = [dictionary.doc2bow(doc, allow_update=True) for doc in [context[h:t]]]
                        if mode == 1:
                            word_vec = {}
                            for id, freq in bow_words[0]:
                                try:
                                    word_vec[id] = model[dictionary[id]]
                                except IndexError:
                                    pass
                                except KeyError:
                                    try:
                                        word_vec[id] = model[dictionary[id] + "e"]
                                    except KeyError:
                                        word_vec[id] = np.zeros(300)
                            embeddings.append(tfidf_weigth(0, word_vec,
                                                                TfidfModel(BoW_corpus, smartirs='ntc'),
                                                                bow_words))
                        if mode == 2:
                            bow_words = [dictionary.doc2bow(doc, allow_update=True) for doc in [context]]
                            word_vec = {}
                            for k, freq in bow_words[0]:
                                try:
                                    word_vec[k] = model[dictionary[k]]
                                except IndexError:
                                    pass
                                except KeyError:
                                    try:
                                        word_vec[k] = model[dictionary[k] + "e"]
                                    except KeyError:
                                        word_vec[k] = np.zeros(300)
                            embeddings.append(
                                tfidf_weigth(0, word_vec, TfidfModel(BoW_corpus, smartirs='ntc'), bow_words))
                        new_labels.append(labels.index(sense))
            data[word] = {"x": embeddings, "y": new_labels}
        return data

Bert embedding and dimension reduction

In [ ]:
    def dimension_reduction(vectors,train_X,test_X):
        pca = PCA(n_components=300)
        reducted_vectors = pca.fit_transform(vectors)
        s = 0
        for word in train_X.keys():
            for sense in train_X[word].keys():
                if sense != "pos":
                    e = s + len(train_X[word][sense])
                    train_X[word][sense] = reducted_vectors[s:e]
                    s = e
        for word in test_X.keys():
            for sense in test_X[word].keys():
                if sense != "pos":
                    e = s + len(test_X[word][sense])
                    test_X[word][sense] = reducted_vectors[s:e]
                    s = e
        return train_X,test_X

    def bert_representation(data_set, docs, mode,arr_embedding_vectors):
        bert_embedding = BertEmbedding(max_seq_length=40)
        dictionary = Dictionary(docs)
        if mode == 0:
            for word in data_set.keys():
                for sense in data_set[word].keys():
                    if sense != "pos":
                        context = data_set[word][sense]
                        for word_c in context:
                            str = ""
                            for txt in word_c:
                                str += txt + " "
                            result = bert_embedding([str.strip(" ")])
                            index = word_c.index(word)
                            arr_embedding_vectors.append(np.ndarray.flatten(np.array(result[0][1][index])))
        if mode == 3:
            for word in data_set.keys():
                for sense in data_set[word].keys():
                    if sense != "pos":
                        context = data_set[word][sense]
                        for word_c in context:
                            str = "[cls]"
                            for txt in word_c:
                                str += txt + " "
                            result = bert_embedding([str.strip(" ")])
                            index = word_c.index(word)
                            if result[0][0][index] != word_c[index]:
                                print(result[0][0], word_c)
                            arr_embedding_vectors.append(np.ndarray.flatten(np.array(result[0][1][0])))
        # Embedding words in a window
        else:
            for word in data_set.keys():
                for sense in data_set[word].keys():
                    print(word)
                    if sense != "pos":
                        context = data_set[word][sense]
                        for word_c in context:
                            str = " "
                            for txt in word_c:
                                str += txt + " "
                            result = bert_embedding([str.strip()])
                            if mode == 1:
                                index = word_c.index(word)
                                head = index - 3
                                tail = index + 4
                                if index - 3 < 0:
                                    head = 0
                                if index + 4 > len(word_c):
                                    tail = len(word_c)
                                bow_words = [dictionary.doc2bow(doc, allow_update=True) for doc in [word_c[head:tail]]]
                                vector_dict = {}
                                for id, freq in bow_words[0]:
                                    vector = np.ndarray.flatten(np.array(result[0][1][word_c.index(dictionary[id])]))
                                    vector_dict[id] = vector[:768]
                                arr_embedding_vectors.append(self.tfidf_weigth(1, vector_dict,
                                                                                    TfidfModel(BoW_corpus,
                                                                                             smartirs='ntc'),
                                                                                    bow_words))
                            if mode == 2:
                                bow_words = [dictionary.doc2bow(doc, allow_update=True) for doc in [word_c]]
                                vector_dict = {}
                                for id, freq in bow_words[0]:
                                    try:
                                        vector = np.ndarray.flatten(np.array(result[0][1][word_c.index(dictionary[id])]))
                                        vector_dict[id] = vector[:768]
                                    except IndexError:
                                        vector = np.ndarray.flatten(np.array(result[0][1][0]))
                                        vector_dict[id] = vector[:768]
                                arr_embedding_vectors.append(self.tfidf_weigth(1, vector_dict,
                                                                                    TfidfModel(BoW_corpus,
                                                                                             smartirs='ntc'),
                                                                                    bow_words))
        return arr_embedding_vectors

main block for creating necessary files

In [ ]:
dict, train, test = load_dict()
data_train, train_all = load_data(train, "drive/MyDrive/NLP/hw3/train.csv")
data_test, test_all = load_data(test, "drive/MyDrive/NLP/hw3/test.csv")
arr_output = []
arr = bert_representation(data_train, train_all, 2,arr_output)
arr = bert_representation(data_test, test_all, 2,arr)
dimension_reduction(arr,train,test)
train = train_xy(dict,train)
test = test_xy(dict,test)
## save bert_embedding vectors 
for word in train.keys():
   pd.DataFrame(train[word]['x']).to_csv("drive/MyDrive/NLP/hw3/train_bert0/"+word+".csv")
   pd.DataFrame(train[word]['y']).to_csv("drive/MyDrive/NLP/hw3/train_bert0/"+word+"_label.csv")
for word in test.keys():
   pd.DataFrame(test[word]['x']).to_csv("drive/MyDrive/NLP/hw3/test_bert0/"+word+".csv")
   pd.DataFrame(test[word]['y']).to_csv("drive/MyDrive/NLP/hw3/test_bert0/"+word+"_label.csv")

In [ ]:
def fetch_data(path1,path2):
  for word in dict_w.keys():
    train_data = pd.read_csv(path1+"/"+word+".csv").to_numpy()
    train_label = pd.read_csv(path1+"/"+word+"_label.csv").to_numpy()[:,1]
    test_data = pd.read_csv(path2+"/"+word+".csv").to_numpy()
    test_label = pd.read_csv(path2+"/"+word+"_label.csv").to_numpy()[:,1]
    train[word] = {"x":train_data,"y":train_label}
    test[word] = {"x":test_data,"y":test_label}

# Part 3(a)

In [ ]:
def train_classifier(mode):
  for word in dict_w.keys():
    if mode == 0:
      lgst_model = LogisticRegression(max_iter=20000,C=0.08)     
      lgst_model.fit(train[word]['x'],train[word]['y'])
      rf_model = RandomForestClassifier(n_estimators=150,max_depth=100)
      rf_model.fit(train[word]['x'],train[word]['y'])
      rf_classifier[word] = rf_model
      lr_classifier[word] = lgst_model
    if mode == 1:
      rf_model = RandomForestClassifier(n_estimators=150,max_depth=100)
      rf_model.fit(train[word]['x'],train[word]['y'])
      rf_classifier1[word] = rf_model
    if mode == 2:
      rf_model = RandomForestClassifier(n_estimators=150,max_depth=100)
      rf_model.fit(train1[word]['x'],train1[word]['y'])
      rf_classifier2[word] = rf_model
    if mode == 3:
      rf_model = RandomForestClassifier(n_estimators=150,max_depth=100)
      rf_model.fit(train2[word]['x'],train2[word]['y'])
      rf_classifier3[word] = rf_model


def classifier(classifiers):
    POS = {}
    measures = []
    for word in classifiers.keys():
            pred_y = classifiers[word].predict(test[word]['x'])
            if word == 'paper':
              print(test[word]['y'], pred_y)
            accuracy = accuracy_score(test[word]['y'], pred_y)
            f_score = f1_score(test[word]['y'], pred_y, average='micro')
            measures.append([accuracy, f_score])
            if word == 'paper':
              print(measures[-1])
            try:
               POS[dict_w[word]['pos']]["acc"] += accuracy
               POS[dict_w[word]['pos']]["fm"] += f_score
               POS[dict_w[word]['pos']]["c"] += 1
            except KeyError:
              POS[dict_w[word]['pos']] ={"acc":accuracy,
                                    "fm":f_score,
                                    "c":1}
    for p in POS.keys():
        POS[p]["acc"] /= POS[p]["c"]
        POS[p]["fm"] /= POS[p]["c"]
    print(POS)

# part 3(b)

In [ ]:
def enssemble(classifier1,classifier2,classifier3):
    POS = {}
    measures = []
    for word in classifier1.keys(): 
            pred_y1 = classifier1[word].predict(test[word]['x'])
            pred_y2 = classifier2[word].predict(test1[word]['x'])
            pred_y3 = classifier3[word].predict(test2[word]['x'])
            pred_y = predict(pred_y1, pred_y2, pred_y3)
            accuracy = accuracy_score(test[word]['y'], pred_y)
            f_score = f1_score(test[word]['y'], pred_y, average='micro')
            measures.append([float("{0:.4f}".format(accuracy)), float("{0:.4f}".format(f_score))])
            try:
               POS[dict_w[word]['pos']]["acc"] += accuracy
               POS[dict_w[word]['pos']]["fm"] += f_score
               POS[dict_w[word]['pos']]["c"] += 1
            except KeyError:
              POS[dict_w[word]['pos']] ={"acc":accuracy,
                                    "fm":f_score,
                                    "c":1}
    for p in POS.keys():
        POS[p]["acc"] /= POS[p]["c"]
        POS[p]["fm"] /= POS[p]["c"]
    print(POS)

def predict(first,second,third):
  y = []
  for l1,l2,l3 in zip(first,second,third):
       y.append(np.bincount([l1,l2,l3]).argmax())
  return y

# part 3(c)

In [ ]:
def transformed_data(pred1,pred2,pred3):
  pred_y = []
  for y1,y2,y3 in zip(pred1,pred2,pred3):
       pred_y.append([y1,y2,y3])
  return pred_y

def enssemble_v2(classifier1,classifier2,classifier3):
    POS = {}
    measures = []
    for word in classifier1.keys(): 
            pred_y1 = classifier1[word].predict(test[word]['x'])
            pred_y2 = classifier2[word].predict(test1[word]['x'])
            pred_y3 = classifier3[word].predict(test2[word]['x'])
            test_x = transformed_data(pred_y1, pred_y2, pred_y3)
            pred_y1 = classifier1[word].predict(train[word]['x'])
            pred_y2 = classifier2[word].predict(train1[word]['x'])
            pred_y3 = classifier3[word].predict(train2[word]['x'])
            train_x = transformed_data(pred_y1, pred_y2, pred_y3)  
            rf_model = RandomForestClassifier(n_estimators=150,max_depth=100)
            rf_model.fit(train_x, train[word]['y'])
            predict_label =rf_model.predict(test_x) 
            accuracy = accuracy_score(test[word]['y'],predict_label)
            f_score = f1_score(test[word]['y'], predict_label, average='micro')
            measures.append([float("{0:.4f}".format(accuracy)), float("{0:.4f}".format(f_score))])
            try:
               POS[dict_w[word]['pos']]["acc"] += accuracy
               POS[dict_w[word]['pos']]["fm"] += f_score
               POS[dict_w[word]['pos']]["c"] += 1
            except KeyError:
              POS[dict_w[word]['pos']] ={"acc":accuracy,
                                    "fm":f_score,
                                    "c":1}
    for p in POS.keys():
        POS[p]["acc"] /= POS[p]["c"]
        POS[p]["fm"] /= POS[p]["c"]

    print(POS)


main block for part 3(a)

In [ ]:
dict_w, _,_ = load_dict() 
data_train, train_all = load_data(train, "drive/MyDrive/NLP/hw3/train.csv")
data_test, test_all = load_data(test, "drive/MyDrive/NLP/hw3/test.csv")
model = gensim.models.KeyedVectors.load_word2vec_format(
     'drive/MyDrive/NLP/hw3/GoogleNews-vectors-negative300.bin.gz',binary=True)
## bert_vectors without context + logistic regression & random forest classifier
train = {}
test = {}
fetch_data("drive/MyDrive/NLP/hw3/train_bert0","drive/MyDrive/NLP/hw3/test_bert0")
lr_classifier = {}
rf_classifier = {}
train_classifier(0)
classifier(lr_classifier) 
classifier(rf_classifier) 
#3 bert_vectors with 3 length window + logistic regression & random forest classifier
train = {}
test = {}
fetch_data("drive/MyDrive/NLP/hw3/train_bert1","drive/MyDrive/NLP/hw3/test_bert1")
lr_classifier = {}
rf_classifier = {}
train_classifier(0)
classifier(lr_classifier) 
classifier(rf_classifier) 
# bert_vectors with 9 length window + logistic regression & random forest classifier
train = {}
test = {}
fetch_data("drive/MyDrive/NLP/hw3/train_bert2","drive/MyDrive/NLP/hw3/test_bert2")
lr_classifier = {}
rf_classifier = {}
train_classifier(0)
classifier(lr_classifier) 
classifier(rf_classifier) 
# word2vec with 3 length window + logistic regression & random forest classifier
train = word2vec_representation(data_train,train_all,1,dict_w)
test = word2vec_representation(data_test,test_all,1,dict_w)
lr_classifier = {}
rf_classifier = {}
train_classifier(0)
classifier(lr_classifier) 
classifier(rf_classifier) 
# word2vec with 9 length window + logistic regression & random forest classifier
train = word2vec_representation(data_train,train_all,2,dict_w)
test = word2vec_representation(data_test,test_all,2,dict_w)
lr_classifier = {}
rf_classifier = {}
train_classifier(0)
classifier(lr_classifier) 
classifier(rf_classifier) 

main block for part 3(b)

In [ ]:
rf_classifier1 = {}
rf_classifier2 = {}
rf_classifier3 = {}
train = {}
test = {}
fetch_data("drive/MyDrive/NLP/hw3/train_bert0","drive/MyDrive/NLP/hw3/test_bert0")
train1 = word2vec_representation(data_train,wsd.train_all,1,dict_w)
test1 = word2vec_representation(data_test,wsd.test_all,1,dict_w)
train2 = word2vec_representation(data_train,wsd.train_all,2,dict_w)
test2 = word2vec_representation(data_test,wsd.test_all,2,dict_w)
train_classifier(1)
train_classifier(2)
train_classifier(3)
enssemble(rf_classifier1,rf_classifier2,rf_classifier3)

main block for part 3(c)

In [ ]:
enssemble_v2(rf_classifier1,rf_classifier2,rf_classifier3)